In [ ]:
from keras import initializers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import Dropout
from keras.layers import concatenate

In [ ]:
def create_mlp(dim, class_names):
	# define our MLP network
  model = Sequential()
  # hidden layer with 40 nodes, input = 5
  model.add(Dense(40, input_dim=dim, activation="relu"))
  # output layer with 15 nodes
  model.add(Dense(len(class_names), activation="softmax"))
  # return our model
  return model

In [ ]:
def create_mlp_multi(dim):
	# define our MLP network
  model = Sequential()
  # hidden layer with 40 nodes, input = 5
  model.add(Dense(40, input_dim=dim, activation="relu"))
  # return our model
  return model

In [ ]:
def create_cnn(num_tokens, embedding_dim, embedding_matrix, class_names, filter):
  # define the model input
  int_sequences_input = Input(shape=(None,), dtype="int64")
  # embedding layer
  embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=initializers.Constant(embedding_matrix),
    trainable=False,
  )
  embedded_sequences = embedding_layer(int_sequences_input)
  # CONV => RELU => MP => CONV => RELU => MP => CONV => RELU => GMP
  x = Conv1D(filter, 5, activation="relu")(embedded_sequences)
  x = MaxPooling1D(5)(x)
  x = Conv1D(filter, 5, activation="relu")(x)
  x = MaxPooling1D(5)(x)
  x = Conv1D(filter, 5, activation="relu")(x)
  x = GlobalMaxPooling1D()(x)
  # FC => RELU => DROPOUT
  x = Dense(filter, activation="relu")(x)
  x = Dropout(0.5)(x)
  # FC => SOFTMAX
  preds = Dense(len(class_names), activation="softmax")(x)
  # construct the CNN
  model = Model(int_sequences_input, preds)
  # return the CNN
  return model

In [ ]:
def create_cnn_multi(num_tokens, embedding_dim, embedding_matrix, class_names, filter):
  # define the model input
  int_sequences_input = Input(shape=(None,), dtype="int64")
  # embedding layer
  embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=initializers.Constant(embedding_matrix),
    trainable=False,
  )
  embedded_sequences = embedding_layer(int_sequences_input)
  # CONV => RELU => MP => CONV => RELU => MP => CONV => RELU => GMP
  x = Conv1D(filter, 5, activation="relu")(embedded_sequences)
  x = MaxPooling1D(5)(x)
  x = Conv1D(filter, 5, activation="relu")(x)
  x = MaxPooling1D(5)(x)
  x = Conv1D(filter, 5, activation="relu")(x)
  x = GlobalMaxPooling1D()(x)
  # FC => RELU => DROPOUT
  x = Dense(filter, activation="relu")(x)
  x = Dropout(0.5)(x)
  # construct the CNN
  model = Model(int_sequences_input, x)
  # return the CNN
  return model

In [ ]:
def create_multi(mlp, cnn, class_names):
  # create the input to our final set of layers as the *output* of both
  # the MLP and CNN
  combinedInput = concatenate([mlp.output, cnn.output])
  # FC => SOFTMAX
  preds = Dense(len(class_names), activation="softmax")(combinedInput)
  # our final model will accept categorical/numerical data on the MLP
  # input and descriptions on the CNN input, outputting a single value (the
  # predicted category of the purchase)
  model = Model(inputs=[mlp.input, cnn.input], outputs=preds)
  return model